<a href="https://colab.research.google.com/github/xingchenzhao/study_deep_learning/blob/master/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install d2lzh  # installing d2l
!pip install -U --pre mxnet-cu101mkl  # updating mxnet to at least v1.6

![alt text](https://miro.medium.com/max/2434/1*_rCyzi7fQzc_Q1gCqSLM1g.png)

In [0]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

In [0]:
class Inception(nn.Block):
  def __init__(self, c1, c2, c3, c4, **kwargs):
    super(Inception, self).__init__(**kwargs)

    self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu')
    self.p1_1.

    self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
    self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu')

    self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
    self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu')

    self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
    self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')

  def forward(self, x):
    p1 = self.p1_1(x)
    p2 = self.p2_2(self.p2_1(x))
    p3 = self.p3_2(self.p3_1(x))
    p4 = self.p4_2(self.p4_1(x))
    return nd.concat(p1, p2, p3, p4, dim=1)

Build GoogLeNet Model

In [0]:
b1 = nn.Sequential()
b1.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b2 = nn.Sequential()
b2.add(nn.Conv2D(64, kernel_size=1, activation='relu'),
       nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b3 = nn.Sequential()
b3.add(Inception(64, (96,128), (16,32), 32),
       Inception(128, (128,192), (32,96), 64),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b4 = nn.Sequential()
b4.add(Inception(192, (96,208), (16,48), 64),
       Inception(160, (112,224), (24,64), 64),
       Inception(128, (128,256), (24,64), 64),
       Inception(112, (144,288), (32,64), 64),
       Inception(256, (160,320), (32,128), 128),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b5 = nn.Sequential()
b5.add(Inception(256, (160,320), (32,128),128),
       Inception(384, (192,384), (48,128),128),
       nn.GlobalAvgPool2D())

net = nn.Sequential()
net.add(b1, b2, b3, b4, b5, nn.Dense(10))

In [20]:
X = nd.random.uniform(shape=(1, 1, 96, 96))
net.initialize()
for layer in net:
  X = layer(X)
  print(layer.name, 'output shape:\t',X.shape)

sequential7 output shape:	 (1, 64, 24, 24)
sequential8 output shape:	 (1, 192, 12, 12)
sequential9 output shape:	 (1, 480, 6, 6)
sequential10 output shape:	 (1, 832, 3, 3)
sequential11 output shape:	 (1, 1024, 1, 1)
dense0 output shape:	 (1, 10)


In [23]:
lr, num_epochs, batch_size, ctx = 0.05, 10, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 2.0828, train acc 0.238, test acc 0.550, time 52.3 sec
epoch 2, loss 0.8112, train acc 0.682, test acc 0.785, time 51.6 sec
epoch 3, loss 0.8985, train acc 0.676, test acc 0.755, time 52.0 sec
epoch 4, loss 0.5154, train acc 0.810, test acc 0.833, time 51.9 sec
epoch 5, loss 0.4145, train acc 0.846, test acc 0.852, time 51.8 sec
epoch 6, loss 0.3645, train acc 0.862, test acc 0.860, time 51.9 sec
epoch 7, loss 0.3339, train acc 0.874, test acc 0.882, time 52.2 sec
epoch 8, loss 0.3119, train acc 0.882, test acc 0.889, time 51.4 sec
epoch 9, loss 0.2943, train acc 0.889, test acc 0.879, time 52.0 sec
epoch 10, loss 0.2782, train acc 0.895, test acc 0.891, time 51.9 sec
